<div style="border: 5px solid purple; padding: 10px; margin: 5px">
<b> Svetlana's comment  </b>
    

Hi, my name is Svetlana (https://hub.tripleten.com/u/6dee602c).


Thank you for submitting such an excellent report! It is well-organized, following a logical structure: what was done, the challenges you faced, the decisions you made, and final results. This makes it easy to follow. You don't need to change anything, because it's great! 
    
Good luck! 

</div>

Interconnect Churn Prediction — a practical model to flag at-risk customers: Solution Report

I followed the plan pretty closely. First, I loaded the four CSV files (contract, personal, internet, phone) and joined them on customerID to get one table. I did a quick sanity check (shape, column names, basic missing values, and duplicate IDs) and then created the target the way the brief asks: customers with EndDate == "No" are active (0) and anything else is churn (1). To avoid any leakage from future dates, I computed tenure from the account BeginDate up to a fixed snapshot date (2020-02-01) for everyone who is still active, and from BeginDate to EndDate for those who left. After that, I did light feature engineering: I normalized all the service flags into consistent Yes/No values, built a few compact signals such as is_month_to_month, has_fiber, auto_pay, addon_count, and a simple count of how many services a customer uses. I split the data into train/test with stratification and built a preprocessing pipeline that imputes missing values (median for numeric, most-frequent for categorical), scales numeric features, and one-hot encodes categoricals. Then I trained three models Logistic Regression (baseline), Random Forest, and Gradient Boosting and compared them with cross-validated AUC-ROC. I did a small grid search for Gradient Boosting and evaluated the best model on the test set. Finally, I plotted the ROC curve and used Youden’s J to suggest a working decision threshold.
I skipped two things on purpose: (1) heavy EDA/plotting (I kept it minimal because the rubric is focused on modeling and AUC), and (2) class rebalancing/oversampling. The class imbalance wasn’t extreme here, and the tree-based models handle it reasonably well; I also used class_weight on the linear model. I also avoided any columns that could leak future info (for example, EndDate itself or engineered versions of it), and I dropped IDs.

I did run into a few real problems on the way. The first was date handling when building tenure mixing strings and timestamps will crash math on some rows. I fixed that by forcing all dates through pd.to_datetime(..., errors="coerce"), using the snapshot date for active customers, and taking min(parsed_end, snapshot) so nobody “looks into the future.” The second was NaNs inside the pipeline, which gave me nan CV scores at first. Adding SimpleImputer to both the numeric and categorical branches fixed that. The third was a version mismatch with OneHotEncoder (sparse_output only exists in newer scikit-learn). I made a tiny helper that tries the new argument and falls back to the old sparse flag, so it runs on JupyterHub regardless of the exact version. Lastly, the classic Telco gotcha is TotalCharges being read as text; I coerced it to numeric and it was fine.

The key pieces that made the model work were: (1) leak-safe tenure and target creation, (2) a robust ColumnTransformer (imputers + scaling + OHE with handle_unknown="ignore"), (3) stratified CV for reliable AUC estimates, (4) a small but meaningful grid search on Gradient Boosting, and (5) setting a business-friendly threshold from the ROC curve instead of defaulting to 0.50. The feature importance from the final tree model also passed the sniff test: tenure_months, Type_Month-to-month, TotalCharges, MonthlyCharges, and service-protection items like OnlineSecurity_No/TechSupport_No all ranked high, which matches the story we expect for churn.

My final model is a GradientBoostingClassifier (best grid around n_estimators=250, learning_rate=0.1, max_depth=3). On the hold-out test set it achieved AUC-ROC ≈ 0.86 and Accuracy ≈ 0.85. That lands in the 5.5 SP band of the rubric (0.85–0.88). Using Youden’s J, the suggested decision threshold came out around 0.30, which gives a nicer recall for churners without blowing up false positives too much. Overall, the pipeline runs end-to-end on JupyterHub, avoids leakage, and the metrics are stable across CV and test, so I’m confident in the result.